## Launching Spark and HDFS

In [ ]:
# import the jupyter notebook importer
import import_ipynb
from data603__ import SparkLauncher
from data603__ import HDFS

spark = SparkLauncher.get_spark_session(pack_venv = False)
hdfs = HDFS.get_hdfs()

importing Jupyter notebook from /scratch/data603/rgite1/data603__/SparkLauncher.ipynb
importing Jupyter notebook from /scratch/data603/rgite1/data603__/HDFS.ipynb
Setting Environment Variables
Creating Spark Configuration
Creating Spark Session: rgite1_data603_spark_session


## Importing neccesary libraries

In [ ]:
from pyspark.sql.functions import lit
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import col
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from PIL import Image
import io
from io import BytesIO

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

## Reading the parquet file which contains the Images with the 'Human Face' label

In [ ]:
face_datafile = spark.read.parquet('/user/bthota1/image_face.parquet')

In [ ]:
face_datafile.count()

100

## Print Schema looks nice, we have all the columns to procced to implement openCV

In [ ]:
face_datafile.printSchema()

root
 |-- ImageID: string (nullable = true)
 |-- Data: binary (nullable = true)
 |-- XMin: double (nullable = true)
 |-- XMax: double (nullable = true)
 |-- YMin: double (nullable = true)
 |-- YMax: double (nullable = true)



## In order to read the images and to save them we have to convert our dataframe into toPandas()

In [ ]:
face_data = face_datafile.toPandas()

## Saving the images into the Directory

In [ ]:
for i in range(100):
    save_Face_Image = Image.open(io.BytesIO(face_data['Data'][i]))
    file = face_data['ImageID'][i] +'.jpg' 
    save_Face_Image.save('Images/'+file)

## we are reading name of all the images we saved in the directory from above code

In [ ]:
from os import listdir
from os.path import isfile, join
mypath='Images/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

## Implementing OpenCV

### haarcascade_frontalface_default.xml provides all the features of a face, with the help of this xml openCV indentifies a face in the Image

In [ ]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

### Looping through all the images we saved, converting them into GrayScale and with the help of detectMultiScale, we are going to detect the face in the image and save it in a list

In [ ]:
faceCountList=[]
for fileName in onlyfiles:
    img = cv2.imread('Images/'+fileName)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces_detected = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
    if len(faces_detected) == 0:
        faceCountList.append(0)
    else:
        faceCountList.append(faces_detected.shape[0])

### Calculating the sum of all the values in out list, This will give us the total number of faces we found in the images we saved

In [ ]:
sum=0
for i in faceCountList:
    sum=sum+i

In [ ]:
print('Total sum of Faces in 100 images :' + str(sum))

Total sum of Faces in  :246
